In [4]:
import os
import pandas as pd
import numpy as np
import torch
from torchvision import transforms

from utils.utils import load_config, create_folder, scale_coords
from utils.plots import *
from dataloaders import *
from models import *
from validate import *

from utils.plots import *
from utils.utils import scale_coords, resize_landmarks


import pickle
import cv2

from insightface.app import FaceAnalysis
from tqdm import tqdm

from XAI import *

from tqdm import tqdm
import re
import gc

from torch.utils.data import DataLoader, TensorDataset

import matplotlib

import warnings
warnings.filterwarnings("ignore")

c:\Users\leonardo\anaconda3\envs\doutorado\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
path_icopevid = 'Datasets\\Originais\\iCOPE\\iCOPEvid'
path_icopevid_frames = 'Datasets\\Originais\\iCOPE\\iCOPEvid\\all_frames'

# Face Detection


In [ ]:
retinaface = FaceAnalysis(allowed_modules=['detection','landmark_2d_106'], providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
retinaface.prepare(ctx_id=0, det_size=(640, 640)) 

In [ ]:
for video_path in tqdm(os.listdir(path_icopevid)):
    if video_path.endswith('mp4'):
        video_name = video_path.split('.mp4')[0]

        if "Pain" in video_name or "Rest" in video_name:

            create_folder(os.path.join(path_icopevid_frames, video_name))

            video = cv2.VideoCapture(os.path.join(path_icopevid, video_path))

            idx = 0
            while True:
                ret, frame = video.read()

                if not ret:
                    break
                
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                try:
                    faces = retinaface.get(frame_rgb)[0]
                    bbox = faces['bbox'].astype('int')
                    bbox[bbox < 0] = 0
                    keypoints = faces['kps'].astype('int')
                    landmarks = faces['landmark_2d_106'].astype('int')

                    # Scale the landmarks based on the previous bbox, so it matches the facial image shape
                    scaled_landmarks = [scale_coords(x, y, bbox) for x, y in landmarks]

                    x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
                    cropped_face = frame[y1:y2, x1:x2]
                    cropped_face = cv2.resize(cropped_face, (512, 512))

                    # Resize the landmarks to match the new image size
                    resized_landmarks = resize_landmarks(np.array(scaled_landmarks), cropped_face.shape[:2], (512, 512))

                    cv2.imwrite(os.path.join(path_icopevid_frames, 
                                            video_name, 
                                            video_name+"_"+f'{idx:04d}'+'.jpg'), cropped_face)


                    create_folder(os.path.join(path_icopevid_frames, 
                                            video_name,
                                            'landmarks'))

                    with open(os.path.join(path_icopevid_frames, 
                                            video_name, 
                                            'landmarks',
                                            video_name+"_"+f'{idx:04d}'+'.pkl'), 'wb') as f:
                        pickle.dump(resized_landmarks, f)

                except IndexError:
                    cv2.imwrite(os.path.join(path_icopevid_frames, 
                                            video_name, 
                                            video_name+"_"+f'{idx:04d}'+'_blank'+'.jpg'), np.zeros((512,512,1)))
                    pass

                idx += 1
            video.release()

# Audio Extraction

In [ ]:
from moviepy.editor import VideoFileClip
import numpy as np

In [ ]:
for video_path in tqdm(os.listdir(path_icopevid)):
    if video_path.endswith('mp4'):
        video_name = video_path.split('.mp4')[0]

        if "Pain" in video_name or "Rest" in video_name:

            # Load the video using MoviePy
            video_clip = VideoFileClip(os.path.join(path_icopevid, video_path), audio_fps=48000)

            # Extract audio from the video
            audio_signal = video_clip.audio

            # Convert audio signal to a numpy array
            audio_array = np.array(audio_signal.to_soundarray())

            with open(os.path.join(path_icopevid_frames,video_name,f'audio_signal.pkl'), 'wb') as f:
                pickle.dump(audio_array, f)

            # Close the video clip
            video_clip.close()


# Creating the predictions dataframe

## Predictions with MCDP

In [ ]:
from os import path


model_name = "VGGFace_FINAL"
final_model = '\\20250826_1353_VGGFace'

path_experiments = 'experiments\\' + model_name
path_modell = path_experiments + final_model 

create_folder(os.path.join(path_experiments,"icopevid"))

device = "cuda"

In [ ]:
from uncertainty.MCDropout import MCDropout
from dataloaders import iCOPEVidDataset

In [ ]:
p = 0.5
reps = 50

In [ ]:
results_video = dict()
for video_path in tqdm(os.listdir(path_icopevid_frames)):
    if not video_path.endswith('.pkl'):
        results_folds = dict()
        # Para cada video iremos rodar os modelos de cada fold e salvar os resultados  
        # Carregar modelo e config.yaml
        path_model = os.path.join(path_modell, 'Model','best_model.pt')
        path_yaml = os.path.join(path_modell, 'Model','config.yaml')

        # Carrego o .yaml para pegar o path do test, ou seja, o fold que foi utilizado
        config = load_config(path_yaml)
        test_path = config['path_test']
        fold = test_path.split('\\')[-2]
        
        # Carrego o .yaml para pegar o path do test, ou seja, o fold que foi utilizado
        if "NCNN" in path_model:
            model = NCNN().to(device)
            name = "NCNN"
        elif "VGGFace" in path_model:
            model = VGGFace().to(device)
            name = "VGGFace"
        elif "ViT" in path_model:
            model = ViT().to(device)
            name = "ViT"
        
        # Carrego o modelo
        model.eval()
        model.load_state_dict(torch.load(path_model))
        model = model.to(device)
        model = MCDropout(model, p=p)
        
        # Iteração sobre os dados salvando os probs, preds e labels
        probs_list = torch.empty(0, device=device)

        dataset = iCOPEVidDataset(os.path.join(path_icopevid_frames,video_path), name)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

        with torch.no_grad():
            for batch in dataloader:
                img_batch, blanks, _ = batch

                probs = model.predict(img_batch.to(device), reps)

                probs[blanks] = np.nan

                probs_list = torch.cat([probs_list, probs])
                    
        results_folds[fold] = probs_list.cpu().numpy()

        gc.collect()
        torch.cuda.empty_cache()
                
        results_video[video_path] = results_folds

with open(os.path.join(path_experiments,"icopevid",f'results_icopevid_MCDP_{reps}_{p}.pkl'), 'wb') as f:
    pickle.dump(results_video, f) 

## Predictions Ensemble

In [ ]:
from os import path


model_name = "ViT_B_32_ENSEMBLE_FINAL\\ensemble_9"
final_model = '\\20250826_1529_ViT'

path_experiments = 'experiments\\' + model_name
path_modell = path_experiments + final_model 

create_folder(os.path.join(path_experiments,"icopevid"))

device = "cuda"

In [ ]:
results_video = dict()
for video_path in tqdm(os.listdir(path_icopevid_frames)):
    if not video_path.endswith('.pkl'):
        results_folds = dict()
        # Para cada video iremos rodar os modelos de cada fold e salvar os resultados  
        # Carregar modelo e config.yaml
        path_model = os.path.join(path_modell, 'Model','best_model.pt')
        path_yaml = os.path.join(path_modell, 'Model','config.yaml')

        # Carrego o .yaml para pegar o path do test, ou seja, o fold que foi utilizado
        config = load_config(path_yaml)
        test_path = config['path_test']
        fold = test_path.split('\\')[-2]
        
        # Carrego o .yaml para pegar o path do test, ou seja, o fold que foi utilizado
        if "NCNN" in path_model:
            model = NCNN().to(device)
            name = "NCNN"
        elif "VGGFace" in path_model:
            model = VGGFace().to(device)
            name = "VGGFace"
        elif "ViT" in path_model:
            model = ViT().to(device)
            name = "ViT"
        
        # Carrego o modelo
        model.eval()
        model.load_state_dict(torch.load(path_model))
        model = model.to(device)
        
        # Iteração sobre os dados salvando os probs, preds e labels
        probs_list = torch.empty(0, device=device)

        dataset = iCOPEVidDataset(os.path.join(path_icopevid_frames,video_path), name)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

        with torch.no_grad():
            for batch in dataloader:
                img_batch, blanks, _ = batch

                probs = model.predict(img_batch.to(device))

                probs[blanks] = np.nan

                probs_list = torch.cat([probs_list, probs])
                    
        results_folds[fold] = probs_list.cpu().numpy()

        gc.collect()
        torch.cuda.empty_cache()
                
        results_video[video_path] = results_folds

with open(os.path.join(path_experiments,"icopevid",f'results_icopevid.pkl'), 'wb') as f:
    pickle.dump(results_video, f) 

In [ ]:
stacked_results = {}
num_ensembles = 10
base_path = 'experiments\\ViT_B_32_ENSEMBLE_FINAL'
output_file= base_path + "\\icopevid\\results_ensemble_10.pkl"

for i in range(num_ensembles):
    file_path = os.path.join(base_path, f"ensemble_{i}", "icopevid", "results_icopevid.pkl")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing file: {file_path}")

    with open(file_path, "rb") as f:
        results = pickle.load(f)

    for video, inner_dict in results.items():
        if "TrainAll" not in inner_dict:
            raise KeyError(f"'TrainAll' key not found for video {video} in {file_path}")

        probs = np.array(inner_dict["TrainAll"])
        if video not in stacked_results:
            stacked_results[video] = {"TrainAll": []}
        stacked_results[video]["TrainAll"].append(probs)

# Convert lists to stacked numpy arrays (shape: [num_ensembles, ...])
for video in stacked_results:
    stacked_results[video]["TrainAll"] = np.stack(stacked_results[video]["TrainAll"], axis=0)

# Save final dictionary
with open(output_file, "wb") as f:
    pickle.dump(stacked_results, f)

    print(f"✅ Stacked results saved to {output_file}")

# Create XAIs

In [6]:
import torch.nn as nn

def enable_mc_dropout(model: nn.Module, p: float = 0.1):
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.p = p
            m.train()

def welford_update(mean, m2, x, k):
    if mean is None:
        mean = x.astype(np.float32)
        m2   = np.zeros_like(mean, dtype=np.float32)
    else:
        delta = x - mean
        mean += delta / k
        m2   += delta * (x - mean)
    return mean, m2

def welford_finalize(mean, m2, n, unbiased=False):
    denom = (n - 1) if unbiased and n > 1 else n
    std   = np.sqrt(m2 / denom)
    return mean, std

def to_uint8(m): return (np.clip(m, 0, 1)*255).round().astype(np.uint8)

## MCDP

In [ ]:
path_experiments = 'experiments\\ViT_B_32_ENSEMBLE_FINAL'
model_path = '20250826_1353_VGGFace'
model_name = "VGGFace"
device = "cuda"

In [10]:
DROPOUT_P = 0.5
MC_SAMPLES = 50
UNBIASED_STD = False

In [ ]:
# Carregar modelo e config.yaml
path_model = os.path.join(path_experiments, model_path, 'Model', 'best_model.pt')
path_yaml  = os.path.join(path_experiments, model_path, 'Model', 'config.yaml')

# Carrego o .yaml para pegar o path do test (fold)
config    = load_config(path_yaml)
test_path = config['path_test']
fold      = test_path.split('\\')[-2]

# Instancia o modelo e os métodos de atribuição
if "NCNN" in path_experiments:
    model   = NCNN().to(device)
    ig      = IntegratedGradients(model, device=device)
    gradcam = GradCAM(model, model.merge_branch[0], device=device)
elif "VGGFace" in path_experiments:
    model   = VGGFace().to(device)
    ig      = IntegratedGradients(model, device=device)
    gradcam = GradCAM(model, model.VGGFace.features.conv5_3, device=device)
elif "ViT" in path_experiments:
    model   = ViT().to(device)
    ig      = IntegratedGradients(model, device=device)
    gradcam = GradCAM(model, model.ViT.encoder.layers.encoder_layer_11.ln_1,
                      device=device, reshape_transform_ViT=True)
else:
    raise ValueError("Unsupported model in path_experiments")

# Carrega pesos e coloca em eval
model.load_state_dict(torch.load(path_model))
model.eval()
model = model.to(device)


# Habilita MC Dropout
enable_mc_dropout(model, DROPOUT_P)

BATCH_SIZE = 32  # Ajuste conforme a GPU

for video_path in tqdm(os.listdir(path_icopevid_frames)):
    if video_path.endswith('.pkl'):
        continue

    # Pasta de saída para máscaras XAI com MCDP
    mcdp_folder = os.path.join(path_experiments, "icopevid", "xai_masks_MCDP", video_path)
    create_folder(mcdp_folder)

    dataset = iCOPEVidDataset(os.path.join(path_icopevid_frames, video_path), model_name)

    batch_imgs  = []
    batch_files = []

    for img, blank, file_name in dataset:
        if blank:
            continue

        batch_imgs.append(img)
        batch_files.append(file_name)

        if len(batch_imgs) == BATCH_SIZE:
            batch_tensor = torch.stack(batch_imgs, dim=0).to(device)

            ig_mean = ig_m2 = gc_mean = gc_m2 = None
            for k in range(1, MC_SAMPLES + 1):
                masks_ig_k = ig.attribution_mask(batch_tensor)
                masks_gc_k = gradcam.attribution_mask(batch_tensor)
                ig_mean, ig_m2 = welford_update(ig_mean, ig_m2, masks_ig_k, k)
                gc_mean, gc_m2 = welford_update(gc_mean, gc_m2, masks_gc_k, k)

            masks_ig_mean, masks_ig_std = welford_finalize(ig_mean, ig_m2, MC_SAMPLES, UNBIASED_STD)
            masks_gc_mean, masks_gc_std = welford_finalize(gc_mean, gc_m2, MC_SAMPLES, UNBIASED_STD)

            for i, f_name in enumerate(batch_files):
                base = f_name.replace(".jpg", "")
                out = to_uint8(masks_ig_mean[i])
                np.savez_compressed(os.path.join(mcdp_folder, f"{base}_IG.npz"), mask=out)

                out = to_uint8(masks_ig_std[i])
                np.savez_compressed(os.path.join(mcdp_folder, f"{base}_IG_std.npz"), mask=out)

                out = to_uint8(masks_gc_mean[i])
                np.savez_compressed(os.path.join(mcdp_folder, f"{base}_GC.npz"), mask=out)

                out = to_uint8(masks_gc_std[i])
                np.savez_compressed(os.path.join(mcdp_folder, f"{base}_GC_std.npz"), mask=out)


            batch_imgs  = []
            batch_files = []

    # Processa lote final incompleto
    if len(batch_imgs) > 0:
        batch_tensor = torch.stack(batch_imgs, dim=0).to(device)

        ig_mean = ig_m2 = gc_mean = gc_m2 = None
        for k in range(1, MC_SAMPLES + 1):
            masks_ig_k = ig.attribution_mask(batch_tensor)
            masks_gc_k = gradcam.attribution_mask(batch_tensor)
            ig_mean, ig_m2 = welford_update(ig_mean, ig_m2, masks_ig_k, k)
            gc_mean, gc_m2 = welford_update(gc_mean, gc_m2, masks_gc_k, k)

        masks_ig_mean, masks_ig_std = welford_finalize(ig_mean, ig_m2, MC_SAMPLES, UNBIASED_STD)
        masks_gc_mean, masks_gc_std = welford_finalize(gc_mean, gc_m2, MC_SAMPLES, UNBIASED_STD)

        for i, f_name in enumerate(batch_files):
            base = f_name.replace(".jpg", "")
            out = to_uint8(masks_ig_mean[i])
            np.savez_compressed(os.path.join(mcdp_folder, f"{base}_IG.npz"), mask=out)

            out = to_uint8(masks_ig_std[i])
            np.savez_compressed(os.path.join(mcdp_folder, f"{base}_IG_std.npz"), mask=out)

            out = to_uint8(masks_gc_mean[i])
            np.savez_compressed(os.path.join(mcdp_folder, f"{base}_GC.npz"), mask=out)

            out = to_uint8(masks_gc_std[i])
            np.savez_compressed(os.path.join(mcdp_folder, f"{base}_GC_std.npz"), mask=out)

    # Limpeza por vídeo
    del dataset, batch_imgs, batch_files
    gc.collect()
    torch.cuda.empty_cache()


100%|██████████| 120/120 [10:50:13<00:00, 325.11s/it] 


## ENSEMBLES

In [7]:
path_experiments = 'experiments\\ViT_B_32_ENSEMBLE_FINAL'
model_name = "ViT"
device = "cuda"
UNBIASED_STD = False

In [8]:
import os, re, gc, pickle
import torch
from tqdm import tqdm

# Find ensemble folders (accepts "ensemble_*" and common misspelling "ensenble_*")
ensemble_dirs = sorted(
    d for d in os.listdir(path_experiments)
    if os.path.isdir(os.path.join(path_experiments, d)) and re.match(r"ense[nm]ble_\d+$", d)
)
if not ensemble_dirs:
    raise RuntimeError(f"No ensemble_* folders found under {path_experiments}")

def find_model_dir(ens_root: str) -> str:
    # Prefer exactly one extra folder level: <ensemble>/<something>/Model
    try:
        for d in os.listdir(ens_root):
            candidate = os.path.join(ens_root, d, 'Model')
            if os.path.isdir(candidate):
                return candidate
    except FileNotFoundError:
        pass

    # Fallback to old layout: <ensemble>/Model
    direct = os.path.join(ens_root, 'Model')
    if os.path.isdir(direct):
        return direct

    # Last resort: search recursively
    for root, dirs, files in os.walk(ens_root):
        if os.path.basename(root) == 'Model':
            return root

    raise RuntimeError(f"Could not find a 'Model' directory under {ens_root}")

# Precompute Model dirs for all ensembles
ensemble_model_dirs = [
    find_model_dir(os.path.join(path_experiments, ens_name)) for ens_name in ensemble_dirs
]

# Load config from the first ensemble if you need the fold info
path_yaml = os.path.join(ensemble_model_dirs[0], 'config.yaml')
config    = load_config(path_yaml)
test_path = config['path_test']
fold      = test_path.split('\\')[-2]

# Instantiate model + attribution methods once; we will swap weights per ensemble
if "NCNN" in path_experiments:
    model   = NCNN().to(device)
    ig      = IntegratedGradients(model, device=device)
    gradcam = GradCAM(model, model.merge_branch[0], device=device)
elif "VGGFace" in path_experiments:
    model   = VGGFace().to(device)
    ig      = IntegratedGradients(model, device=device)
    gradcam = GradCAM(model, model.VGGFace.features.conv5_3, device=device)
elif "ViT" in path_experiments:
    model   = ViT().to(device)
    ig      = IntegratedGradients(model, device=device)
    gradcam = GradCAM(model, model.ViT.encoder.layers.encoder_layer_11.ln_1,
                      device=device, reshape_transform_ViT=True)
else:
    raise ValueError("Unsupported model in path_experiments")

def process_batch(batch_imgs, batch_files, out_folder):
    if not batch_imgs:
        return

    batch_tensor = torch.stack(batch_imgs, dim=0).to(device)

    # Aggregate across ensembles (single deterministic pass per ensemble; no MC Dropout)
    ens_mean_ig = ens_m2_ig = ens_mean_gc = ens_m2_gc = None

    for e_idx, model_dir in enumerate(ensemble_model_dirs, 1):
        path_model_e = os.path.join(model_dir, 'best_model.pt')

        # Load this ensemble member
        state = torch.load(path_model_e, map_location=device)
        model.load_state_dict(state)
        model.eval().to(device)  # dropout/bn in eval; no MCDP

        # Compute attributions once per ensemble
        masks_ig = ig.attribution_mask(batch_tensor)
        masks_gc = gradcam.attribution_mask(batch_tensor)

        # Update ensemble-level Welford accumulators
        ens_mean_ig, ens_m2_ig = welford_update(ens_mean_ig, ens_m2_ig, masks_ig, e_idx)
        ens_mean_gc, ens_m2_gc = welford_update(ens_mean_gc, ens_m2_gc, masks_gc, e_idx)

    # Final mean + std across ensembles
    final_ig_mean, final_ig_std = welford_finalize(ens_mean_ig, ens_m2_ig, len(ensemble_dirs), UNBIASED_STD)
    final_gc_mean, final_gc_std = welford_finalize(ens_mean_gc, ens_m2_gc, len(ensemble_dirs), UNBIASED_STD)

    # Save only final results
    for i, f_name in enumerate(batch_files):
        base = f_name.replace(".jpg", "")

        out = to_uint8(final_ig_mean[i])
        np.savez_compressed(os.path.join(out_folder, f"{base}_IG.npz"), mask=out)

        out = to_uint8(final_ig_std[i])
        np.savez_compressed(os.path.join(out_folder, f"{base}_IG_std.npz"), mask=out)

        out = to_uint8(final_gc_mean[i])
        np.savez_compressed(os.path.join(out_folder, f"{base}_GC.npz"), mask=out)

        out = to_uint8(final_gc_std[i])
        np.savez_compressed(os.path.join(out_folder, f"{base}_GC_std.npz"), mask=out)

    torch.cuda.empty_cache()

BATCH_SIZE = 32  # Ajuste conforme a GPU

for video_path in tqdm(os.listdir(path_icopevid_frames)[111:]):
    if video_path.endswith('.pkl'):
        continue

    # Output folder for final ensemble XAI (mean + std)
    out_folder = os.path.join(path_experiments, "icopevid", "xai_masks_ENSEMBLE", video_path)
    create_folder(out_folder)

    dataset = iCOPEVidDataset(os.path.join(path_icopevid_frames, video_path), model_name)

    batch_imgs  = []
    batch_files = []

    for img, blank, file_name in dataset:
        if blank:
            continue

        batch_imgs.append(img)
        batch_files.append(file_name)

        if len(batch_imgs) == BATCH_SIZE:
            process_batch(batch_imgs, batch_files, out_folder)
            batch_imgs  = []
            batch_files = []

    # Tail: process last partial batch
    process_batch(batch_imgs, batch_files, out_folder)

    # Cleanup per video
    del dataset, batch_imgs, batch_files
    gc.collect()
    torch.cuda.empty_cache()


100%|██████████| 9/9 [09:12<00:00, 61.42s/it]


## Vizu

In [9]:
from concurrent.futures import ThreadPoolExecutor
import os, gc, pickle
import numpy as np
import cv2
import matplotlib

# Single colormap matching your green→yellow→red
_CMAP = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green", "yellow", "red"])

def _colorize(mask_norm, cmap=_CMAP):
    # mask_norm: float [0,1], shape (H, W)
    mask_norm = np.asarray(mask_norm, dtype=np.float32)
    mask_norm = np.clip(mask_norm, 0.0, 1.0)
    rgba = cmap(mask_norm)                  # (H, W, 4), float [0,1]
    return (rgba[..., :3] * 255).astype(np.uint8)  # (H, W, 3), uint8

def _maybe_resize_2d(arr, size):
    # size is (W, H). If arr is scalar, return as-is. If 2D, resize to (W,H) if needed.
    if np.isscalar(arr):
        return arr
    arr = np.asarray(arr)
    if arr.ndim == 2 and (arr.shape[1] != size[0] or arr.shape[0] != size[1]):
        arr = cv2.resize(arr, size, interpolation=cv2.INTER_LINEAR)
    return arr

def _blend_rgb(base_rgb, heat_rgb, alpha):
    # base_rgb/heat_rgb: uint8 (H, W, 3). alpha: scalar or (H, W)
    if np.isscalar(alpha):
        a = float(alpha)
        return cv2.addWeighted(heat_rgb, a, base_rgb, 1.0 - a, 0.0)
    a = np.clip(alpha, 0.0, 1.0).astype(np.float32)
    a = a[..., None]  # (H, W, 1)
    out = base_rgb.astype(np.float32) * (1.0 - a) + heat_rgb.astype(np.float32) * a
    return out.astype(np.uint8)

def create_xais(images_dir, xai_dir, model, num_workers=None):
    # Resolve output size based on model family
    size = (224, 224) if ("VGGFace" in model or "ViT" in model) else (120, 120)

    os.makedirs(xai_dir, exist_ok=True)

    # Collect frame files to process
    frame_files = [
        f for f in os.listdir(images_dir)
        if ("blank" not in f) and f.lower().endswith((".jpg", ".png"))
    ]

    def _paths(file_name):
        # Matches your naming (if .png, it keeps .png in stem, as in your code)
        return (
            os.path.join(xai_dir, file_name.replace('.jpg', '_GC.npz')),
            os.path.join(xai_dir, file_name.replace('.jpg', '_IG.npz')),
            os.path.join(xai_dir, file_name.replace('.jpg', '_GC_std.npz')),
            os.path.join(xai_dir, file_name.replace('.jpg', '_IG_std.npz')),
        )

    def _out_paths(file_name):
        return (
            os.path.join(xai_dir, file_name.replace('.jpg', '._GC.jpg')),
            os.path.join(xai_dir, file_name.replace('.jpg', '._IG.jpg')),
            os.path.join(xai_dir, file_name.replace('.jpg', '._GC_std.jpg')),
            os.path.join(xai_dir, file_name.replace('.jpg', '._IG_std.jpg')),
        )

    def _process_one(file_name):
        try:
            # Read frame
            img_bgr = cv2.imread(os.path.join(images_dir, file_name))
            if img_bgr is None:
                return False
            img_rgb = cv2.cvtColor(cv2.resize(img_bgr, size), cv2.COLOR_BGR2RGB)

            # Load masks
            gc_pkl, ig_pkl, gc_std_pkl, ig_std_pkl = _paths(file_name)
            mask_gc = np.load(gc_pkl)['mask'].astype(np.float32) / 255.0
            mask_ig = np.load(ig_pkl)['mask'].astype(np.float32) / 255.0
            mask_gc_std = np.load(gc_std_pkl)['mask'].astype(np.float32) / 255.0
            mask_ig_std = np.load(ig_std_pkl)['mask'].astype(np.float32) / 255.0

            # Normalize and alphas using your existing helper
            mask_gc_norm, alpha_gc       = attribution_mask_processing(mask_gc)
            mask_ig_norm, alpha_ig       = attribution_mask_processing(mask_ig)
            mask_gc_std_norm, alpha_gc_s = attribution_mask_processing(mask_gc_std)
            mask_ig_std_norm, alpha_ig_s = attribution_mask_processing(mask_ig_std)

            # Ensure masks/alphas match the target size
            mask_gc_norm   = _maybe_resize_2d(mask_gc_norm, size)
            mask_ig_norm   = _maybe_resize_2d(mask_ig_norm, size)
            mask_gc_std_n  = _maybe_resize_2d(mask_gc_std_norm, size)
            mask_ig_std_n  = _maybe_resize_2d(mask_ig_std_norm, size)
            alpha_gc       = _maybe_resize_2d(alpha_gc, size)
            alpha_ig       = _maybe_resize_2d(alpha_ig, size)
            alpha_gc_s     = _maybe_resize_2d(alpha_gc_s, size)
            alpha_ig_s     = _maybe_resize_2d(alpha_ig_s, size)

            # Colorize and blend
            heat_gc     = _colorize(mask_gc_norm)
            heat_ig     = _colorize(mask_ig_norm)
            heat_gc_std = _colorize(mask_gc_std_n)
            heat_ig_std = _colorize(mask_ig_std_n)

            out_gc     = _blend_rgb(img_rgb, heat_gc, alpha_gc)
            out_ig     = _blend_rgb(img_rgb, heat_ig, alpha_ig)
            out_gc_std = _blend_rgb(img_rgb, heat_gc_std, alpha_gc_s)
            out_ig_std = _blend_rgb(img_rgb, heat_ig_std, alpha_ig_s)

            # Save
            out_gc_path, out_ig_path, out_gc_s_path, out_ig_s_path = _out_paths(file_name)
            cv2.imwrite(out_gc_path,     cv2.cvtColor(out_gc, cv2.COLOR_RGB2BGR))
            cv2.imwrite(out_ig_path,     cv2.cvtColor(out_ig, cv2.COLOR_RGB2BGR))
            cv2.imwrite(out_gc_s_path,   cv2.cvtColor(out_gc_std, cv2.COLOR_RGB2BGR))
            cv2.imwrite(out_ig_s_path,   cv2.cvtColor(out_ig_std, cv2.COLOR_RGB2BGR))
            return True
        except Exception:
            # Optional: log per-file errors for debugging
            # print(f"[warn] {file_name}: {e}")
            return False

    max_workers = num_workers or (os.cpu_count() or 4)
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        list(ex.map(_process_one, frame_files))  # simple parallel map

    gc.collect()


In [10]:
for video_path in tqdm(os.listdir(path_experiments + "\\icopevid\\xai_masks_ENSEMBLE")):
    if not video_path.endswith(".pkl"):
        create_xais(
            os.path.join(path_icopevid_frames, video_path),
            os.path.join(path_experiments, "icopevid", "xai_masks_ENSEMBLE", video_path),
            model_name,
            num_workers=8  # tune 4–12 based on your disk/CPU
        )


100%|██████████| 120/120 [2:15:19<00:00, 67.67s/it] 


# Pain Signal Visualization

In [ ]:
from validate import validation_metrics
from scipy import stats
import matplotlib
import glob
from scipy.stats import entropy
from scipy import integrate
from scipy.signal import butter, lfilter, freqz, filtfilt

In [ ]:

def butter_lowpass_filter(data, cutoff=2, fs=30, order=2):
    normal_cutoff = cutoff / (0.5 * fs)
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

def get_hist(signal):
    return np.histogram(signal, bins=np.linspace(0.0, 1.0, 11))

def get_probs(signal):
    hist, _ = get_hist(signal)
    return hist / len(signal)

def get_entropy(signal):
    pk = get_probs(signal)
    return entropy(pk, base=2)

def get_entropy_curve(signal):
    hist, bin_edges = get_hist(signal)
    pk = hist / len(signal)
    entropies = []
    for x in signal:
        idx = np.digitize(x, bin_edges, right=True)
        pkx = pk[idx-1]
        entropies.append(-((pkx * np.log(pkx)) / np.log(2)))
    return entropies

def get_auc(signal):
    return integrate.trapezoid(signal) / len(signal)

def get_auc_curve(signal):
    window_size = 10
    step = 1
    auc_curve = []
    for x in range(0, len(signal)-window_size, step):
        window = signal[x:x+window_size]
        auc_curve.append(get_auc(window))
    return auc_curve

def interp_curve(signal):
    time = np.linspace(0,len(signal),len(signal))
    # Find indices of missing values
    missing_indices = np.isnan(signal)
    # Find indices of non-missing values
    valid_indices = ~missing_indices
    # Perform linear interpolation
    interpolated_values = np.interp(time[missing_indices], time[valid_indices], signal[valid_indices])
    # Replace missing values with interpolated values
    signal[missing_indices] = interpolated_values

    return signal

def fill_nan(signal):
    # Find indices of missing values
    missing_indices = np.isnan(signal)
    # Find indices of non-missing values
    valid_indices = ~missing_indices
    # Perform linear interpolation
    mean = np.mean(signal[valid_indices])
    # Replace missing values with interpolated values
    signal[missing_indices] = mean

    return signal


def peak_to_peak(signal):
    return (np.abs(np.max(signal)) - np.abs(np.min(signal)))

def derivative(signal, power=False):
    if power:
        return np.power(np.diff(signal),2)
    else:
        return np.diff(signal)
    
def RMS(signal):
    return np.sqrt(np.mean(np.square(signal)))

def thresh_crossing(signal, thresh=0.5):
    return len(np.where(np.diff(np.sign(signal-thresh)))[0]) 

In [ ]:
model = "VGGFace"

In [ ]:
with open(os.path.join(path_experiments,'icopevid',f'results_icopevid_MCDP_50_0.5.pkl'), 'rb') as f:
    results_video = pickle.load(f)

In [ ]:
mcdp = True

## MCDP predictions

In [ ]:
labels = []
preds = []
probs = []

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green","yellow","red"])
plt.style.use('utils\plotstyle.mplstyle')

if "NCNN" in model:
    theta_1 = 0.542
    theta_3 = 0.047
    lower_tresh = 0.1
    upper_tresh = 0.8
elif "VGGNB" in model:
    theta_1 = 0.494
    theta_3 = 0.0447
    lower_tresh = 0.2
    upper_tresh = 0.8
else:
    theta_1 = 0.431


for video in tqdm(results_video.keys()):

    full_frames_path = os.path.join(path_icopevid_frames,video)
    img_files = os.listdir(full_frames_path)
    frame_seq = []

    gc_frame_seq = []
    gc_alpha_seq = []

    ig_frame_seq = []
    ig_alpha_seq= []

    ar_frame_seq = []
    ar_alpha_seq= []

    for i in range(0, len(img_files), 30):
        if img_files[i].endswith('.jpg'):
            img = cv2.imread(os.path.join(full_frames_path, img_files[i]))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256,256))
            frame_seq.append(img/255)

            try:
                mask_gc = cv2.imread(os.path.join(full_frames_path,f"GC_{model}", img_files[i]))
                mask_gc = cv2.cvtColor(mask_gc, cv2.COLOR_BGR2RGB)
                mask_gc = cv2.resize(mask_gc, (256,256))
            except:
                mask_gc = np.zeros((256,256,3))

            try:    
                mask_ig = cv2.imread(os.path.join(full_frames_path,f"IG_{model}", img_files[i]))
                mask_ig = cv2.cvtColor(mask_ig, cv2.COLOR_BGR2RGB)
                mask_ig = cv2.resize(mask_ig, (256,256))
            except:
                mask_ig = np.zeros((256,256,3))

            gc_frame_seq.append(mask_gc/255)
            ig_frame_seq.append(mask_ig/255)

    frame_seq = np.hstack(frame_seq)


    gc_seq = np.hstack(gc_frame_seq)
    ig_seq = np.hstack(ig_frame_seq)

    frame_seq = np.vstack((frame_seq, gc_seq, ig_seq))
   
    if "Pain" in video:
        labels.append(1)
    else:
        labels.append(0)

    if mcdp:
        mean_probs = []
        std_probs = []
        for fold in list(results_video[video].keys()):
            for array in results_video[video][fold]:
                array = np.asarray(array)
                if not np.isnan(array).any():
                    mean_probs.append(array.mean())
                    std_probs.append(array.std())
                else:
                    mean_probs.append(np.nan)
                    std_probs.append(np.nan)

        mean_probs = interp_curve(np.asarray(mean_probs))
        std_probs = interp_curve(np.asarray(std_probs))
        
        ma_window = 30
        ma_probs = moving_average(mean_probs, ma_window)
        std_ma_probs = moving_average(std_probs, ma_window)
    
    else:
        stacked_probs = []
        for fold in list(results_video[video].keys()):
            stacked_probs.append(interp_curve(results_video[video][fold]))

        stacked_probs = np.array(stacked_probs)
        mean_probs = stacked_probs.mean(axis=0)
        
        ma_window = 30
        ma_probs = moving_average(mean_probs, ma_window)
        
    preds.append((ma_probs.mean() >= theta_1).astype('int'))
    probs.append(ma_probs.mean())

    time = np.linspace(0,20,len(ma_probs))

    #idx_precise = (ma_probs <= lower_tresh) & (ma_probs >= upper_tresh)
    #idx_certain = std_ma_probs <= theta_3

    #ma_probs_precision = ma_probs[idx_precision]
    #std_ma_precision = std_ma_probs[idx_precision]
    #time_precision = time[idx_precision]

    #ma_probs_imprecision = ma_probs[~idx_precision]
    #std_ma_imprecision = std_ma_probs[~idx_precision]
    #time_imprecision = time[~idx_precision]

    #ma_probs_precision = np.ma.masked_where(std_ma_probs > theta_3, ma_probs) # é ao contratrio essa função
    #ma_probs_imprecision = np.ma.masked_where(std_ma_probs <= theta_3, ma_probs)

    #std_ma_precision = np.ma.masked_where(std_ma_probs > theta_3, std_ma_probs) # é ao contratrio essa função
    #std_ma_imprecision = np.ma.masked_where(std_ma_probs <= theta_3, std_ma_probs)

    idx = np.where(np.diff(np.sign(ma_probs-theta_1)))[0]
   
    plt.figure(figsize=(15,6))

    plt.subplot(2,1,1)

    plt.plot(time, ma_probs, 'k')
    #plt.plot(time, ma_probs_precision, 'k', time, ma_probs_imprecision, 'r')
    #plt.plot(time_audio, (audio_signal*0.8)+0.5, 'b', alpha=0.2)
    #plt.fill_between(x=time, y1=ma_probs+std_ma_probs, y2=ma_probs-std_ma_probs, alpha=0.2, color="#000000", edgecolor="none")
    #plt.fill_between(x=time, y1=ma_probs_precision+std_ma_precision, y2=ma_probs_precision-std_ma_precision, alpha=0.2, color="#000000", edgecolor="none")
    #plt.fill_between(x=time, y1=ma_probs_imprecision+std_ma_imprecision, y2=ma_probs_imprecision-std_ma_imprecision, alpha=0.2, color="red", edgecolor="none")

    plt.ylim([-0.05,1.05])
    plt.xlim([-0.05,20])

    label = "Pain" if "Pain" in video else "No Pain"

    temp_1 = "Pain" if ma_probs.mean() >= theta_1 else "No Pain"

    #temp_3 = "Certo" if std_ma_probs.mean() <= theta_3 else "Incerto"

    #plt.title(f"Classe real = {label} / $\\hat{{p}}$ = {ma_probs.mean():.4f} $\\rightarrow$ {temp_1} / $\\hat{{\\sigma}}$ = {std_ma_probs.mean():.4f} $\\rightarrow$ {temp_3} / $H$ = {get_entropy(ma_probs):.4f} / Pontos de Inflexão =  {thresh_crossing(ma_probs, theta_1)}", fontsize=20)
    plt.title(f"Model = {model} / Label = {label} / $\\hat{{p}}$ = {ma_probs.mean():.2f} $\\rightarrow$ {temp_1} / Entropy = {get_entropy(ma_probs):.2f} / Crossings = {thresh_crossing(ma_probs, theta_1)}") #, fontsize=20
    plt.ylabel("Pain Probability")
    plt.xlabel("Time [s]")

    plt.plot(time[idx],[theta_1]*len(idx), 'ro')

    plt.hlines(y=theta_1, xmin=0,xmax=20, linestyle=":", color='#9e9e9e')
    #plt.text(20.1, theta_1, "$\\theta_{{1}}$", fontsize=20)

    #plt.hlines(y=lower_tresh, xmin=0,xmax=20, linestyle=":", color='#9e9e9e')
    #plt.text(20.1, lower_tresh, "$\\theta_{{2}}$", fontsize=20)

    #plt.hlines(y=upper_tresh, xmin=0,xmax=20, linestyle=":", color='#9e9e9e')
    #plt.text(20.1, upper_tresh, "$\\theta_{{2}}$", fontsize=20)

    #plt.text(0.5, 0.8, f"$\\hat{{p}}$ = {ma_probs.mean():.4f}\n"+
                       #f"$\\hat{{\\sigma}}$ = {std_ma_probs.mean():.4f}\n"+
                       #f"$H$ = {get_entropy(ma_probs):.4f}\n"+
                       #f"Pontos de Inflexão =  {thresh_crossing(ma_probs, theta_1)}")

    #plt.legend(loc='upper right')

    #plt.subplot(5,1,2)
    #entropies = get_entropy_curve(ma_probs)
    #plt.plot(entropies, label=f'Entropy: {get_entropy(ma_probs):.4f}', color="#000000")
    #plt.legend(loc='upper right')
    #plt.xlim([-0.05,600])
    #plt.ylim([-0.05,1.05])

    #plt.subplot(6,1,3)
    #auc_curve = get_auc_curve(ma_probs)
    #plt.plot(auc_curve, label=f'AUC: {get_auc(ma_probs):.4f}', color="#000000")
    #plt.legend(loc='upper right')
    #plt.xlim([-0.05,600])
    #plt.ylim([-0.05,1.05])

    plt.subplot(2,1,2)
    plt.imshow(frame_seq)
    plt.axis('off')

    plt.text(-50, 125, 'Frames', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 375, 'GC', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 625, 'IG', horizontalalignment='center', verticalalignment='center', rotation=90)


    #plt.subplot(3,1,3)
    #plt.specgram(audio_signal, Fs=48000, NFFT=512, cmap='jet')

    #plt.subplot(4,1,3)
    #plt.imshow(frame_seq)
    #plt.imshow(gc_seq, cmap=cmap, alpha=gc_alpha_seq)
    #plt.axis('off')

    #plt.subplot(4,1,4)
    #plt.imshow(frame_seq)
    #plt.imshow(ig_seq, cmap=cmap, alpha=ig_alpha_seq)
    #plt.axis('off')

    #plt.show()
    plt.savefig(os.path.join("C:\\Users\\leonardo\\Desktop\\icopevid_results", f'{video}_{model}.pdf'), dpi=300, bbox_inches='tight')
    plt.close()

gc.collect()


print(validation_metrics(np.array(preds), np.array(probs), np.array(labels)))

## TEST ALL PLOTS

In [ ]:
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green","yellow","red"])
plt.style.use('utils\plotstyle.mplstyle')

def stack_XAI(img_files, model):
    gc_frame_seq = []
    ig_frame_seq = []

    for i in range(0, len(img_files), 30):
        if img_files[i].endswith('.jpg'):

            try:
                mask_gc = cv2.imread(os.path.join(full_frames_path,f"GC_{model}", img_files[i]))
                mask_gc = cv2.cvtColor(mask_gc, cv2.COLOR_BGR2RGB)
                mask_gc = cv2.resize(mask_gc, (256,256))
            except:
                mask_gc = np.zeros((256,256,3))

            try:    
                mask_ig = cv2.imread(os.path.join(full_frames_path,f"IG_{model}", img_files[i]))
                mask_ig = cv2.cvtColor(mask_ig, cv2.COLOR_BGR2RGB)
                mask_ig = cv2.resize(mask_ig, (256,256))
            except:
                mask_ig = np.zeros((256,256,3))

            gc_frame_seq.append(mask_gc/255)
            ig_frame_seq.append(mask_ig/255)

    gc_seq = np.hstack(gc_frame_seq)
    ig_seq = np.hstack(ig_frame_seq)

    return np.vstack((gc_seq, ig_seq))

with open(os.path.join(path_icopevid_frames,f'results_NCNN_test.pkl'), 'rb') as f:
    results_video_NCNN = pickle.load(f)

with open(os.path.join(path_icopevid_frames,f'results_VGGNB_test.pkl'), 'rb') as f:
    results_video_VGGNB = pickle.load(f)

with open(os.path.join(path_icopevid_frames,f'results_ViTNB_test.pkl'), 'rb') as f:
    results_video_ViTNB = pickle.load(f)

for video in results_video_NCNN.keys():

    full_frames_path = os.path.join(path_icopevid_frames,video)
    img_files = os.listdir(full_frames_path)[5:] # MELHORAR ISSO, USEI PARA FICAR IGUAL AO PAPER
    frame_seq = []

    for i in range(0, len(img_files), 30):
        if img_files[i].endswith('.jpg'):
            img = cv2.imread(os.path.join(full_frames_path, img_files[i]))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256,256))
            frame_seq.append(img/255)

    frame_seq = np.hstack(frame_seq)
   
    VGGNB_stack_xai = stack_XAI(img_files, 'VGGNB_test')
    NCNN_stack_xai = stack_XAI(img_files,'NCNN_test')
    ViTNB_stack_xai = stack_XAI(img_files,'ViTNB_test')

    plt.figure(figsize=(20,12))
    plt.subplot(5,1,1)

    for results_video ,name in zip([results_video_VGGNB, results_video_NCNN, results_video_ViTNB],['VGG-Face', 'N-CNN', "ViT-B/16"]):

        stacked_probs = []
        for fold in list(results_video[video].keys()):
            if "N-CNN" in name and fold == "9":
                stacked_probs.append(interp_curve(results_video[video][fold]))
            elif "VGG" in name and fold == "7":
                stacked_probs.append(interp_curve(results_video[video][fold]))
            elif "ViT" in name and fold == "8":
                stacked_probs.append(interp_curve(results_video[video][fold]))

        stacked_probs = np.array(stacked_probs)
        mean_probs = stacked_probs.mean(axis=0)
        
        ma_window = 30
        ma_probs = moving_average(mean_probs, ma_window)

        time = np.linspace(0,20,len(ma_probs))

        

        if "N-CNN" in name:
            theta_1 = 0.542
            color = '#FF9500'
            marker = 'o'
            temp_1 = "Pain" if ma_probs.mean() >= theta_1 else "No Pain"
            legenda = f"{name} / $\\hat{{p}}$ = {ma_probs.mean():.2f} $\\rightarrow$ {temp_1} / Entropy = {get_entropy(ma_probs):.2f} / Crossings = {thresh_crossing(ma_probs, theta_1)}"
        elif "VGG" in name:
            theta_1 = 0.494
            color = '#0C5DA5'
            marker = 's'
            temp_1 = "Pain" if ma_probs.mean() >= theta_1 else "No Pain"
            legenda = f"{name} / $\\hat{{p}}$ = {ma_probs.mean():.2f} $\\rightarrow$ {temp_1} / Entropy = {get_entropy(ma_probs):.2f} / Crossings = {thresh_crossing(ma_probs, theta_1)}"
        else:
            theta_1 = 0.431
            color = '#00B945'
            marker = 'P'
            temp_1 = "Pain" if ma_probs.mean() >= theta_1 else "No Pain"
            legenda = f"{name} $\\text{{}}$ / $\\hat{{p}}$ = {ma_probs.mean():.2f} $\\rightarrow$ {temp_1} / Entropy = {get_entropy(ma_probs):.2f} / Crossings = {thresh_crossing(ma_probs, theta_1)}"
        
        plt.plot(time, ma_probs, label=legenda, linewidth=2)

        idx = np.where(np.diff(np.sign(ma_probs-theta_1)))[0]
        plt.scatter(time[idx],[theta_1]*len(idx), color=color, marker=marker, edgecolor='k', s=50, zorder=10, clip_on=False)

    plt.ylim([-0.05,1.05])
    plt.xlim([-0.05,20])

    label = "Pain" if "Pain" in video else "No Pain"

    plt.ylabel("Pain Probability")
    plt.xlabel("Time [s]")


    #plt.hlines(y=0.5, xmin=0,xmax=20, linestyle=":", color='#9e9e9e')

    plt.legend(title=f"Ground-Truth Label = {label}", loc='upper left', bbox_to_anchor=(-0.05, 1.4), frameon=False, ncol=3)
    
    plt.subplot(5,1,2)
    plt.imshow(frame_seq)
    plt.axis('off')
    plt.text(-50, 125, 'Frames', horizontalalignment='center', verticalalignment='center', rotation=90)

    plt.subplot(5,1,3)
    plt.imshow(VGGNB_stack_xai)
    plt.axis('off')
    plt.text(-150, 250, 'VGG-Face', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 125, 'GC', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 375, 'IG', horizontalalignment='center', verticalalignment='center', rotation=90)

    plt.subplot(5,1,4)
    plt.imshow(NCNN_stack_xai)
    plt.axis('off')
    plt.text(-150, 250, 'N-CNN', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 125, 'GC', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 375, 'IG', horizontalalignment='center', verticalalignment='center', rotation=90)

    plt.subplot(5,1,5)
    plt.imshow(ViTNB_stack_xai)
    plt.axis('off')
    plt.text(-150, 250, 'ViT-B/16', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 125, 'GC', horizontalalignment='center', verticalalignment='center', rotation=90)
    plt.text(-50, 375, 'IG', horizontalalignment='center', verticalalignment='center', rotation=90)

    plt.subplots_adjust(wspace=0, hspace=0)

    plt.savefig(os.path.join("C:\\Users\\leonardo\\Desktop\\icopevid_results", f'{video}_ALL.pdf'), dpi=100, bbox_inches='tight')
    plt.close()

    gc.collect()



In [ ]:

gc.collect()

# Analysis

In [ ]:
model = "VGGNB_test"

In [ ]:
with open(os.path.join(path_icopevid_frames,f'results_{model}_MCDP.pkl'), 'rb') as f:
    results_video = pickle.load(f)

if "NCNN" in model:
    theta_1 = 0.542
    theta_3 = 0.047
    lower_tresh = 0.1
    upper_tresh = 0.8
elif "VGGNB" in model:
    theta_1 = 0.494
    theta_3 = 0.0447
    lower_tresh = 0.2
    upper_tresh = 0.8
else:
    theta_1 = 0.431

In [ ]:
all_probs = []
all_entropy = []
all_crossings = []
all_preds = []
all_curves = []

labels = []

best_fold = '9'

for video in results_video.keys():
    # se MCDP
    mean_probs = []
    for array in results_video[video][best_fold]:
        array = np.asarray(array)
        if not np.isnan(array).any():
            mean_probs.append(array.mean())
        else:
            mean_probs.append(np.nan)

    mean_probs = np.asarray(mean_probs)
    
    interpoleted_curve = interp_curve(mean_probs)
    filtered_curve = moving_average(interpoleted_curve, 30)
    
    # se normal
    #plt.figure(figsize=(20,5))
    #interpoleted_curve = interp_curve(results_video[video][best_fold])
    #filtered_curve = moving_average(interpoleted_curve, 30)
    #plt.plot(filtered_curve)
    #plt.ylim([0,1.01])
    #plt.show()

    all_curves.append(filtered_curve)
    all_probs.append(filtered_curve.mean())
    all_preds.append(filtered_curve.mean() >= theta_1)
    all_entropy.append(get_entropy(filtered_curve))
    all_crossings.append(thresh_crossing(filtered_curve, theta_1))
    
    if "Pain" in video:
        labels.append(1)
    else:
        labels.append(0)

all_probs = np.array(all_probs)
all_entropy = np.array(all_entropy)
all_crossings = np.array(all_crossings)
all_preds = np.array(all_preds)

In [ ]:
metrics = validation_metrics(all_preds, all_probs, np.array(labels))
accuracy = metrics['Accuracy']
f1 = metrics['F1 Score']
precision = metrics['Precision']
sensitivity = metrics['Sensitivity']
specificity = metrics['Specificity']
auc = metrics['AUC']


In [ ]:
metrics

In [ ]:
dataframe = pd.DataFrame({'video':results_video.keys(),'probs':all_probs, 'entropy':all_entropy, 'crossings':all_crossings, 'labels':labels, 'preds': all_preds, 'curves':all_curves})

In [ ]:
dataframe.head()

## Agrupamento dos sinais

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
X = dataframe[['entropy','crossings']]

In [ ]:
X.shape

In [ ]:
X

In [ ]:
inertia =[] 
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(X)
    inertia.append(kmeans.inertia_)

plt.plot(range(1,10),inertia)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto")
kmeans.fit(X)

dataframe['kmeans'] = kmeans.labels_

for videoname, kmeans_label in zip(dataframe['video_name'], dataframe['kmeans']):
    for filename in os.listdir('C:\\Users\\leonardo\\Desktop\\icopevid results\\NCNN\\'):
        if videoname in filename:
            os.rename(os.path.join('C:\\Users\\leonardo\\Desktop\\icopevid results\\NCNN\\',filename),os.path.join(f'C:\\Users\\leonardo\\Desktop\\icopevid results\\NCNN\\{kmeans_label}',filename))
            break

In [ ]:
stable = dataframe[dataframe['kmeans']==0]
irregular = dataframe[dataframe['kmeans']==2]
unstable = dataframe[dataframe['kmeans']==1]

for y in [stable, irregular, unstable]:
    print(f"{y['probs'].mean():.4f};{y['probs'].std():.4f}\n{y['entropy'].mean():.4f};{y['entropy'].std():.4f}\n{np.median(y['crossings']):.4f};{y['crossings'].std():.4f}\n".replace('.',','))

In [ ]:
time = np.linspace(0,20,571)

In [ ]:
def get_delta_t(signal, theta_1):

    time = np.linspace(0,20,len(signal))

    crossings = np.insert(np.where(np.diff(np.sign(signal-theta_1)))[0], 0, 0)
    crossings = np.insert(crossings, len(crossings), len(signal)-1)

    delta_t_pain = []
    delta_t_nopain = []

    for i in range(len(crossings)-1):
        delta_t = time[crossings[i+1]] - time[crossings[i]]
        avg_prob_delta_t = signal[crossings[i]:crossings[i+1]].mean()

        if avg_prob_delta_t >= theta_1:
            delta_t_pain.append(delta_t)
        else:
            delta_t_nopain.append(delta_t)

    return delta_t_pain, delta_t_nopain


### Pain Signal 0

In [ ]:
stable.describe()

In [ ]:
stable['labels'].value_counts()

In [ ]:
stable['preds'].value_counts()

In [ ]:
metrics = validation_metrics(stable['preds'], stable['probs'], stable['labels'])

for values in metrics.values():
    print(f"{values:.4f}".replace('.',","))

In [ ]:
plt.figure(figsize=(20,6))

k_pain = np.array(stable[stable['labels']==1]['curves'])
k_nopain = np.array(stable[stable['labels']==0]['curves'])

mean_pain = np.mean(k_pain,axis=0)
mean_nopain = np.mean(k_nopain,axis=0)

for k in k_pain:
    plt.plot(time, k, 'r', alpha=0.1)

for k in k_nopain:
    plt.plot(time, k, 'b', alpha=0.1)

plt.plot(time, mean_pain, 'r', label='Pain')
plt.plot(time, mean_nopain, 'b', label='Non-pain')

plt.legend()
plt.title('Stable pain signals')
plt.xlabel('Time [s]')
plt.ylabel('Predicted Pain Probability')
plt.legend(loc="lower right")
plt.savefig(f"stable_signal_avg_{model}.pdf", dpi=150, bbox_inches='tight')

In [ ]:
for prediction, k_signal in zip(["PAIN", "NO-PAIN"],[k_pain, k_nopain]):

    delta_t_pain_k = []
    delta_t_nopain_k = []

    total_pain_events = []
    total_nopain_events = []

    for signal in k_signal:
        delta_t_pain, delta_t_nopain = get_delta_t(signal, theta_1)
        
        delta_t_pain_k.extend(delta_t_pain)
        delta_t_nopain_k.extend(delta_t_nopain)

        total_pain_events.append(len(delta_t_pain))
        total_nopain_events.append(len(delta_t_nopain))


    delta_t_pain_k = np.array(delta_t_pain_k)
    delta_t_nopain_k = np.array(delta_t_nopain_k)

    total_pain_events = np.array(total_pain_events)
    total_nopain_events = np.array(total_nopain_events)


    print(f"Label {prediction}: Painful Event duration {delta_t_pain_k.mean():.4f} ± {delta_t_pain_k.std():.2f}, # Occurances: {total_pain_events.mean():.4f} ± {total_pain_events.std():.4f}")
    print(f"Label {prediction}: Non-Painful Event duration {delta_t_nopain_k.mean():.4f} ± {delta_t_nopain_k.std():.2f} # Occurances: {total_nopain_events.mean():.4f} ± {total_nopain_events.std():.4f}")
    print()

### Pain Signal 1

In [ ]:
irregular.describe()

In [ ]:
irregular['labels'].value_counts()

In [ ]:
irregular['preds'].value_counts()

In [ ]:
metrics = (validation_metrics(irregular['preds'], irregular['probs'], irregular['labels']))

for values in metrics.values():
    print(f"{values:.4f}".replace('.',","))

In [ ]:
plt.figure(figsize=(20,6))

k_pain = np.array(irregular[irregular['labels']==1]['curves'])
k_nopain = np.array(irregular[irregular['labels']==0]['curves'])

mean_pain = np.mean(k_pain,axis=0)
mean_nopain = np.mean(k_nopain,axis=0)

for k in k_pain:
    plt.plot(time, k, 'r', alpha=0.1)

for k in k_nopain:
    plt.plot(time, k, 'b', alpha=0.1)

plt.plot(time, mean_pain, 'r', label='Pain')
plt.plot(time, mean_nopain, 'b', label='Non-pain')

plt.legend()
plt.title('Irregular pain signals')
plt.xlabel('Time [s]')
plt.ylabel('Predicted Pain Probability')
plt.legend(loc="lower right")
plt.savefig(f"irregular_signal_avg_{model}.pdf", dpi=150, bbox_inches='tight')

In [ ]:
for prediction, k_signal in zip(["PAIN", "NO-PAIN"],[k_pain, k_nopain]):

    delta_t_pain_k = []
    delta_t_nopain_k = []

    total_pain_events = []
    total_nopain_events = []

    for signal in k_signal:
        delta_t_pain, delta_t_nopain = get_delta_t(signal, theta_1)
        
        delta_t_pain_k.extend(delta_t_pain)
        delta_t_nopain_k.extend(delta_t_nopain)

        total_pain_events.append(len(delta_t_pain))
        total_nopain_events.append(len(delta_t_nopain))


    delta_t_pain_k = np.array(delta_t_pain_k)
    delta_t_nopain_k = np.array(delta_t_nopain_k)

    total_pain_events = np.array(total_pain_events)
    total_nopain_events = np.array(total_nopain_events)


    print(f"Label {prediction}: Painful Event duration {delta_t_pain_k.mean():.4f} ± {delta_t_pain_k.std():.2f}, # Occurances: {total_pain_events.mean():.4f} ± {total_pain_events.std():.4f}")
    print(f"Label {prediction}: Non-Painful Event duration {delta_t_nopain_k.mean():.4f} ± {delta_t_nopain_k.std():.2f} # Occurances: {total_nopain_events.mean():.4f} ± {total_nopain_events.std():.4f}")
    print()

### Pain Signal 2

In [ ]:
unstable.describe()

In [ ]:
unstable['labels'].value_counts()

In [ ]:
unstable['preds'].value_counts()

In [ ]:
metrics = (validation_metrics(unstable['preds'], unstable['probs'], unstable['labels']))

for values in metrics.values():
    print(f"{values:.4f}".replace('.',","))

In [ ]:
plt.figure(figsize=(20,6))

k_pain = np.array(unstable[unstable['labels']==1]['curves'])
k_nopain = np.array(unstable[unstable['labels']==0]['curves'])

mean_pain = np.mean(k_pain,axis=0)
mean_nopain = np.mean(k_nopain,axis=0)

for k in k_pain:
    plt.plot(time, k, 'r', alpha=0.1)

for k in k_nopain:
    plt.plot(time, k, 'b', alpha=0.1)

plt.plot(time, mean_pain, 'r', label='Pain')
plt.plot(time, mean_nopain, 'b', label='Non-pain')

plt.legend()
plt.title('Unstable pain signals')
plt.xlabel('Time [s]')
plt.ylabel('Predicted Pain Probability')
plt.legend(loc="lower right")
plt.savefig(f"unstable_signal_avg_{model}.pdf", dpi=150, bbox_inches='tight')

In [ ]:
for prediction, k_signal in zip(["PAIN", "NO-PAIN"],[k_pain, k_nopain]):

    delta_t_pain_k = []
    delta_t_nopain_k = []

    total_pain_events = []
    total_nopain_events = []

    for signal in k_signal:
        delta_t_pain, delta_t_nopain = get_delta_t(signal, theta_1)
        
        delta_t_pain_k.extend(delta_t_pain)
        delta_t_nopain_k.extend(delta_t_nopain)

        total_pain_events.append(len(delta_t_pain))
        total_nopain_events.append(len(delta_t_nopain))


    delta_t_pain_k = np.array(delta_t_pain_k)
    delta_t_nopain_k = np.array(delta_t_nopain_k)

    total_pain_events = np.array(total_pain_events)
    total_nopain_events = np.array(total_nopain_events)


    print(f"Label {prediction}: Painful Event duration {delta_t_pain_k.mean():.4f} ± {delta_t_pain_k.std():.2f}, # Occurances: {total_pain_events.mean():.4f} ± {total_pain_events.std():.4f}")
    print(f"Label {prediction}: Non-Painful Event duration {delta_t_nopain_k.mean():.4f} ± {delta_t_nopain_k.std():.2f} # Occurances: {total_nopain_events.mean():.4f} ± {total_nopain_events.std():.4f}")
    print()

### CLuster Visualization

In [ ]:
plt.scatter(stable['entropy'], stable['crossings'], marker='s', edgecolors='k', s=65, label='Stable')
plt.scatter(irregular['entropy'], irregular['crossings'], marker='o', edgecolors='k', s=65, label='Irregular')
plt.scatter(unstable['entropy'], unstable['crossings'], marker='P', edgecolors='k', s=65, label='Unstable')

plt.xlim([-0.1,3.5])
plt.ylim([-0.9,10.5])
plt.xlabel('Entropy')
plt.ylabel('Crossings')

#x1 = stable['entropy'].mean()+2*stable['entropy'].std()
#x2 = irregular['entropy'].mean()-2*irregular['entropy'].std()
#x = np.abs(x1-x2)/2

#y1 = irregular['crossings'].mean()+2*irregular['crossings'].std()
#y2 = unstable['crossings'].mean()-2*unstable['crossings'].std()
#y = np.abs(y1-y2)/2

vline = 1.25 if "VGG" in model else 1.75
hline = irregular['crossings'].max()+1

#plt.vlines(vline , ymax=10, ymin=-0.5, color='#8c8c8c', linestyles='--', linewidth=1.2)
#plt.hlines(hline, xmax=3.4, xmin=0, color='#8c8c8c', linestyles='--', linewidth=1.2)

plt.legend()

model_name = "VGG-Face" if "VGG" in model else "N-CNN"
plt.title(f'Pain Signal Types for {model_name}')

plt.savefig(f"plot_entropy_crossings_{model}.pdf", dpi=150, bbox_inches='tight')

In [ ]:
plt.axis('off')

plt.vlines(0.5 , ymax=1, ymin=0, color='#8c8c8c', linestyles='--', linewidth=1.2)
plt.hlines(0.5, xmax=1, xmin=0, color='#8c8c8c', linestyles='--', linewidth=1.2)

plt.text(0.75, 1, 'High Entropy', horizontalalignment='center', verticalalignment='center', weight='bold')
plt.text(0.25, 1, 'Low Entropy', horizontalalignment='center', verticalalignment='center')

plt.text(0, 0.75, 'High Crossings', horizontalalignment='center', verticalalignment='center', rotation=90)
plt.text(0, 0.25, 'Low Crossings', horizontalalignment='center', verticalalignment='center', rotation=90)

plt.text(0.75, 0.75, f"Unstable\n\nAction Needed\n\n Entropy$\ge${vline:.2f}\nCrossings$\ge${hline:.0f}", horizontalalignment='center', verticalalignment='center')
plt.text(0.75, 0.25, f"Irregular\n\nAction Needed\n\n Entropy$\ge${vline:.2f}\nCrossings$<${hline:.0f}", horizontalalignment='center', verticalalignment='center')
plt.text(0.25, 0.75, f"Unsure\n\nAction Needed\n\n Entropy$<${vline:.2f}\nCrossings$\ge${hline:.0f}", horizontalalignment='center', verticalalignment='center')
plt.text(0.25, 0.25, f"Stable\n\nAction Needed\n\n Entropy$<${vline:.2f}\nCrossings$<${hline:.0f}", horizontalalignment='center', verticalalignment='center')

plt.fill_between([0.04, 0.46], 0.04, 0.46, color='#f25757', alpha=0.3)
plt.fill_between([0.04, 0.46], 0.54, 0.96 , color='#f25757', alpha=0.3)
plt.fill_between([0.54, 0.96], 0.04, 0.46 , color='#f25757', alpha=0.3)
plt.fill_between([0.54, 0.96], 0.54, 0.96 , color='#f25757', alpha=0.3)


plt.title(f'For "Pain" predictions of {model_name}')

plt.savefig(f"pain_quadrant_{model}.png", dpi=150, bbox_inches='tight')

In [ ]:
plt.axis('off')

plt.vlines(0.5 , ymax=1, ymin=0, color='#8c8c8c', linestyles='--', linewidth=1.2)
plt.hlines(0.5, xmax=1, xmin=0, color='#8c8c8c', linestyles='--', linewidth=1.2)

plt.text(0.75, 1, 'High Entropy', horizontalalignment='center', verticalalignment='center', weight='bold')
plt.text(0.25, 1, 'Low Entropy', horizontalalignment='center', verticalalignment='center')

plt.text(0, 0.75, 'High Crossings', horizontalalignment='center', verticalalignment='center', rotation=90)
plt.text(0, 0.25, 'Low Crossings', horizontalalignment='center', verticalalignment='center', rotation=90)

plt.text(0.75, 0.75, f"Unstable\n\nAction Needed\n\n Entropy$\ge${vline:.2f}\nCrossings$\ge${hline:.0f}", horizontalalignment='center', verticalalignment='center')
plt.text(0.75, 0.25, f"Irregular\n\nAction Not Needed\n\n Entropy$\ge${vline:.2f}\nCrossings$<${hline:.0f}", horizontalalignment='center', verticalalignment='center')
plt.text(0.25, 0.75, f"Unsure\n\nAction Needed\n\n Entropy$<${vline:.2f}\nCrossings$\ge${hline:.0f}", horizontalalignment='center', verticalalignment='center')
plt.text(0.25, 0.25, f"Stable\n\nAction Not Needed\n\n Entropy$<${vline:.2f}\nCrossings$<${hline:.0f}", horizontalalignment='center', verticalalignment='center')

plt.fill_between([0.04, 0.46], 0.04, 0.46, color='#57F257', alpha=0.3)
plt.fill_between([0.04, 0.46], 0.54, 0.96 , color='#f25757', alpha=0.3)
plt.fill_between([0.54, 0.96], 0.04, 0.46 , color='#57F257', alpha=0.3)
plt.fill_between([0.54, 0.96], 0.54, 0.96 , color='#f25757', alpha=0.3)

plt.title(f'For "No-Pain" predictions of {model_name}')

plt.savefig(f"nopain_quadrant_{model}.png", dpi=150, bbox_inches='tight')